<a href="https://colab.research.google.com/github/CYJ1/NLP/blob/main/NLP_Assignment_1_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**라이브러리 import**

In [1]:
!python --version

Python 3.9.16


In [2]:
  !wget https://www.python.org/ftp/python/3.7.0/Python-3.7.0.tgz
  !tar xvfz Python-3.7.0.tgz
  !Python-3.7.0/configure
  !make
  !sudo make install


  # torchtext 사용을 위해 python downgrade 진행

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_calltips.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_codecontext.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_colorizer.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config_key.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_configdialog.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_debugger.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_delegator.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editmenu.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editor.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_grep.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help_a

In [3]:
!python --version

# 다운그레이드 잘 됐는지 확인

Python 3.7.0


In [4]:
!pip install torch==1.9.0 torchtext==0.10.0

# torchtext.legacy 를 사용하기 위해 torch downgrade 진행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.9.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.9.0 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but 

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator


import spacy
import numpy as np

import random
import math
import time

**랜덤 시드 생성**

In [6]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

**독일, 영어 번역할 것 불러오기**

In [12]:
import spacy
spacy.__version__

'3.5.1'

In [7]:
import spacy.cli
spacy.cli.download("en_core_web_sm")
spacy.cli.download("de_core_news_sm")

# 버전이 올라가면서 그냥 spacy 안에는 데이터가 없음 -> spacy.cli에서 download

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [8]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

**토큰화 하기**

 문장을 단어 단위로 토큰화

In [9]:
def tokenize_de(text):
    """
    독일문장을 단어 단위로 토큰화하여 list에 저장
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    영어문장을 단어 단위로 토큰화하여 list에 저장
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

**Field 정의**

*아직 전처리는 하지 않았음을 명심*


---



init_token = 'sos' # start of sentence

eos_token = 'eos' # end of sentence / 문장이 끝났다는 것까지 학습시킴

lower = 'true' # 전부 소문자로 지정

In [10]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

**데이터 로드**

In [11]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.69MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.83MB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.62MB/s]


**단어 생성**



---

단어 사전 생성 , min_freq는 단어의 최소 등장 빈도 조건


In [12]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [13]:
TRG.vocab.itos[:15]

# 잘 생성됐는지 itos 함수 사용해서 확인

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 'a',
 '.',
 'in',
 'the',
 'on',
 'man',
 'is',
 'and',
 'of',
 'with',
 'woman']

In [14]:
print(len(TRG.vocab))
TRG.vocab.stoi

5893


defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f0c1e14bfa0>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             'a': 4,
             '.': 5,
             'in': 6,
             'the': 7,
             'on': 8,
             'man': 9,
             'is': 10,
             'and': 11,
             'of': 12,
             'with': 13,
             'woman': 14,
             ',': 15,
             'two': 16,
             'are': 17,
             'to': 18,
             'people': 19,
             'at': 20,
             'an': 21,
             'wearing': 22,
             'shirt': 23,
             'young': 24,
             'white': 25,
             'black': 26,
             'his': 27,
             'while': 28,
             'blue': 29,
             'men': 30,
             'red': 31,
             'sitting': 32,
             'girl': 33,
             'boy': 34,
             'dog': 35,
             

In [15]:
SRC.vocab.itos[:15]

# 잘 생성됐는지 itos 함수 사용해서 확인

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '.',
 'ein',
 'einem',
 'in',
 'eine',
 ',',
 'und',
 'mit',
 'auf',
 'mann',
 'einer']

In [16]:
print(len(SRC.vocab))
SRC.vocab.stoi

7853


defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f0dc7905850>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             '.': 4,
             'ein': 5,
             'einem': 6,
             'in': 7,
             'eine': 8,
             ',': 9,
             'und': 10,
             'mit': 11,
             'auf': 12,
             'mann': 13,
             'einer': 14,
             'der': 15,
             'frau': 16,
             'die': 17,
             'zwei': 18,
             'einen': 19,
             'im': 20,
             'an': 21,
             'von': 22,
             'sich': 23,
             'dem': 24,
             'mädchen': 25,
             'junge': 26,
             'vor': 27,
             'zu': 28,
             'steht': 29,
             'männer': 30,
             'sitzt': 31,
             'hund': 32,
             'den': 33,
             'straße': 34,
             'während': 35,
  

**상황에 따라 gpu 혹은 cpu를 사용하도록 장치 지정**

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**iterator 생성**

Iterator를 생성한다. Iterator로는 torchtext의 BucketIterator를 사용하였다.

Iterator는 데이터셋에서 batch_size만큼의 데이터를 샘플링하여 batch를 생성하는 객체로, pytorch에서의 DataLoader와 유사한 기능을 한다.

Iterator 중에서도 BucketIterator는 batch를 생성할 때 비슷한 길이의 데이터(즉, 비슷한 수의 단어로 구성된 문장)를 함께 묶는 iterator이다.

비슷한 길이의 데이터를 묶어 batch로 사용하는 이유는, 하나의 batch에 해당하는 텐서를 구성할 때 길이가 짧은 문장에는 padding token을 더해 길이가 가장 긴 문장과 같은 길이로 만들어 batch를 생성하기 때문이다.

BucketIterator를 사용하면 길이가 비슷한 데이터끼리 batch를 만들기 때문에, 추가되는 padding token의 수를 최소화 할 수 있다.

데이터셋과 배치 사이즈, device를 인자로 입력해 iterator를 생성하였다.
Iterator를 생성할 때 아래와 같이 .splits() 메소드를 사용하면 학습, 검증, 테스트용 iterator를 따로 생성할 필요 없이 한번에 생성할 수 있다.


In [18]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

**Encoder 만들기**

In [19]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden

**Attention Layer**

## Attention matrix의 학습

Attention matrix는 디코더에서 weighted(context vector)를 구할 때, 디코더의 fully connected layer로 확률분포를 계산할 때 관여한다. <br>
따라서 cross entropy loss가 역전파되어 디코더가 업데이트 될 때 attention matrix를 계산하는 attention 클래스의 fully connected layer도 오차 역전파를 통해 loss가 작아지는 방향으로 학습된다.

## Attention 클래스에서 각 텐서의 크기와 값

### hidden
hidden :  torch.Size([1, 512])<br> 
hidden은 디코더의 이전 time step의 hidden state $s_{i-1}$에 해당한다.<br>
값은 디코더에서 확인하였기 때문에 크기만 확인하였다.<br>
encoder_outputs와 함께 fully connected layer에 입력되어 energy, attention을 계산하는데 사용된다.<br><br>
$\alpha_{ij}={exp(e_{ij})}/{\sum_{k=1}^{T_x}exp(e_{ik})}, e_{ij}=a(s_{i-1}, h_j)$이고 분모를 계산하기 위해서는 input sequence의 모든 time step에서의 hidden state와 디코더의 이전 hidden state에 대해 연산이 필요함을 알 수 있다.<br><br>
병렬적으로 계산하기 위해 hidden을 input sequence length만큼 복제하고 인코더의 모든 time step의 hidden state인 encoder_outputs와 concatenate하여 한 번에 fully connected layer에 입력되는 것을 코드에서 확인할 수 있다.<br>
또, fully connected layer에 입력되려면 batch size가 텐서 크기의 첫 번째 차원이 되어야 하므로 permute 함수가 사용되었다.<br>
이렇게 repeat 함수와 permute 함수가 적용되고 나면 hidden의 크기는 아래와 같이 변한다.<br>
torch.Size([1, 20, 512])


### encoder_outputs
encoder_outputs :  torch.Size([1, 20, 1024]) <br>
encoder_outputs는 인코더 RNN의 모든 time step에서의 hidden state(annotation의 시퀀스)에 해당한다.<br>
인코더 클래스의 outputs에 해당하고, 마찬가지로 값은 인코더 클래스에서 확인했기 때문에 크기만 확인하였다.<br>
hidden과 마찬가지로 fully connected layer에 입력되어야 하므로 batch size가 텐서의 첫 번째 차원이 되도록 permute 함수를 사용해 수정되었다.<br>
encoder 클래스의 outputs와 크기를 비교해보면 알 수 있다.<br><br>

### energy, attention의 크기와 값은 hidden, encoder_outputs와 다른 데이터로 확인하여 input sequence length가 다름.

### energy
shape :  torch.Size([1, 11, 512])<br>
tensor([[[-7.7650e-02, -7.3061e-06, -7.7176e-02,  ...,  1.9701e-01,
           9.8334e-03,  2.0467e-01],
         [ 1.0646e-01,  5.6690e-02, -1.3624e-01,  ...,  1.6274e-01,
          -7.8837e-02,  2.0809e-01],
         [ 4.2273e-02,  2.4400e-01, -2.0695e-01,  ..., -1.0549e-01,
           7.1047e-02, -8.8972e-02],
         ...,
         [-9.5785e-03, -4.7416e-02,  9.9112e-02,  ...,  2.0672e-01,
          -1.3523e-01,  4.6240e-02],
         [-1.8347e-01,  1.4507e-03,  6.9254e-03,  ...,  2.9650e-02,
          -1.1803e-01,  2.0337e-02],
         [-5.8472e-02,  1.4870e-01, -9.7312e-02,  ...,  2.0529e-01,
          -6.8486e-02, -1.6317e-01]]], device='cuda:0',
       grad_fn=<TanhBackward0>)<br>

energy의 크기와 값을 확인하였다.<br>
논문에서의 energy는 $e_{ij}=a(s_{i-1}, h_j)$로 구해지는 스칼라이지만, 코드에서의 energy는 512차원의 벡터이다. 코드에서는 $e_{ij}$를 구하는 과정과 softmax를 취해 $\alpha_{ij}$를 구하는 과정을 따로 수행하지 않았고, self.attn과 self.v라는 두 개의 fully connected layer를 통과시키는 것으로 구현했다.<br>
따라서 논문에서의 energy와는 크기가 일치하지 않는다.<br><br>

### attention
shape :  torch.Size([1, 11])<br>
tensor([[ 0.0863,  0.0298, -0.0402,  0.0391,  0.0356, -0.0545,  0.0689, -0.1209,
         -0.0307,  0.1268,  0.1863]], device='cuda:0',
       grad_fn=<SqueezeBackward1>)<br>

사실 코드에서의 attention 텐서가 논문에서의 energy에 가깝다. energy 텐서를 self.v fully connected layer에 입력해 (batch_size, input sequence length) 크기의 텐서로 만든 결과이다.<br>
고정된 i와 모든 input sequence time step j에 대해 energy를 계산하였으므로 크기가 (1,11)이다.<br><br>

### Softmax 이후 attention
shape :  torch.Size([1, 11])
tensor([[0.0959, 0.0906, 0.0845, 0.0914, 0.0911, 0.0833, 0.0942, 0.0779, 0.0853,
         0.0998, 0.1059]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

위에서 구한 attention 텐서에 softmax를 취해 $\alpha_{ij}$를 구한 결과이다.<br>
가중치 $\alpha_{ij}$는 스칼라이지만, 모든 input sequnce time step j에 대해 계산했기 때문에 j개가 존재하여 attention 텐서의 크기는 (batch_size, input sequence length)이다.

In [20]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
      
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
        
        return F.softmax(attention, dim=1)

**Decoder**

In [21]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        return prediction, hidden.squeeze(0)

**Seq2Seq**

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1

        return outputs

**Seq2Seq 모델 학습**

In [23]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

**가중치 초기화**

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# 총 사용가능한 parameter 측정

The model has 20,518,405 trainable parameters


In [26]:
optimizer = optim.Adam(model.parameters())

# optimizer 만들기

In [27]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

# loss function 초기화

**Training 시작**

In [28]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        
        output = model(src, trg)
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

**평가 함수**

In [29]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


# timing function 정의

In [31]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


# 최적화된 validation loss parameter 저장

Epoch: 01 | Time: 1m 30s
	Train Loss: 5.020 | Train PPL: 151.389
	 Val. Loss: 4.813 |  Val. PPL: 123.066
Epoch: 02 | Time: 1m 33s
	Train Loss: 4.125 | Train PPL:  61.884
	 Val. Loss: 4.567 |  Val. PPL:  96.277
Epoch: 03 | Time: 1m 32s
	Train Loss: 3.470 | Train PPL:  32.128
	 Val. Loss: 3.794 |  Val. PPL:  44.422
Epoch: 04 | Time: 1m 32s
	Train Loss: 2.915 | Train PPL:  18.441
	 Val. Loss: 3.421 |  Val. PPL:  30.614
Epoch: 05 | Time: 1m 32s
	Train Loss: 2.523 | Train PPL:  12.465
	 Val. Loss: 3.293 |  Val. PPL:  26.916
Epoch: 06 | Time: 1m 33s
	Train Loss: 2.235 | Train PPL:   9.347
	 Val. Loss: 3.245 |  Val. PPL:  25.658
Epoch: 07 | Time: 1m 32s
	Train Loss: 1.994 | Train PPL:   7.346
	 Val. Loss: 3.161 |  Val. PPL:  23.602
Epoch: 08 | Time: 1m 33s
	Train Loss: 1.776 | Train PPL:   5.904
	 Val. Loss: 3.212 |  Val. PPL:  24.817
Epoch: 09 | Time: 1m 33s
	Train Loss: 1.605 | Train PPL:   4.977
	 Val. Loss: 3.225 |  Val. PPL:  25.155
Epoch: 10 | Time: 1m 32s
	Train Loss: 1.496 | Train PPL

In [32]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')


| Test Loss: 3.164 | Test PPL:  23.660 |


**Translate 함수 구현**

In [33]:
def translate(model, iterator):
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            '''배치의 첫 데이터만을 사용
               vocabulary에서 token의 index로부터 단어를 찾아 출력'''
            if i == 1:
                break
            src = batch.src
            trg = batch.trg
            src, trg = src[:,0], trg[:,0]
            print("Source sentence")
            for token in src:
                print(SRC.vocab.itos[token])
            print("\nTarget sentence")
            for token in trg:
                print(TRG.vocab.itos[token])

            src, trg = src.reshape(-1, 1), trg.reshape(-1, 1)
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            output = output.argmax(dim=1)
            print("\nTranslated sentence")
            for token in output:
                print(TRG.vocab.itos[token])

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

In [34]:
translate(model, test_iterator)

Source sentence
<sos>
zwei
mittelgroße
hunde
laufen
über
den
schnee
.
<eos>

Target sentence
<sos>
two
medium
sized
dogs
run
across
the
snow
.
<eos>
<pad>
<pad>
<pad>

Translated sentence
two
steeplechase
and
dogs
are
running
across
the
snow
.
<eos>
<eos>
.


**구글 번역 함수**

In [38]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [39]:
!pip uninstall googletrans
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 7.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=a4aef727a04084efbd78f92b26a883049e8006c486c87abf64f3ba139de23d2c
  Stored in directory: /root/.cache/pip/wheels/ae/e1/6c/5137bc3f35aa130deea71575e165cc4f4f0680a88f3d90a636
Successfully built googletrans


In [41]:
import googletrans

translator = googletrans.Translator()
inStr = input("\n번역할 독일어 문장 입력 : \n")
outStr = translator.translate(inStr, dest='en')
# 독일어에서 영어로 번역

print(f"\n{inStr} => {outStr.text}")


번역할 독일어 문장 입력 : 
zwei mittelgroße hunde laufen über den schnee

zwei mittelgroße hunde laufen über den schnee => two medium-sized dogs are walking across the snow
